In [1]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.feature_extraction.image import extract_patches_2d
from sklearn.cluster import KMeans
from skimage import feature
import imageio

data_path = '/kaggle/input/cifar10-pngs-in-folders/cifar10/test/'

image_paths = []
count = 0
for folder in os.listdir(data_path):
    data_path = data_path + folder
    for file in os.listdir(data_path):
        file_path = data_path + '/' +file
        image_paths.append(file_path)
    data_path = '/kaggle/input/cifar10-pngs-in-folders/cifar10/test/'

def get_patches(img_path, random_state, patch_size, number_patches,is_query=False):
    """
    Extract patches from an image.

    Parameters:
    - img (numpy.ndarray): Input image data.
    - patch_size (tuple): Size of patches (height, width).
    - n_patches (int): Number of patches to extract.
    - is_query (bool): Whether the image is a query image.

    Returns:
    - patches (numpy.ndarray or None): Extracted patches, or None if extraction fails.
    """
    try:
        img = cv2.imread(img_path)
        if is_query:
            height, width = img.shape[:2]
            img_with_zeros = cv2.copyMakeBorder(img, 0, 4, 0, 0, cv2.BORDER_CONSTANT, value=(255, 255, 255))
            img = img_with_zeros[4:, :]
        if img is None:
            raise ValueError("Unable to load image: {}".format(img_path))
        patch = extract_patches_2d(img, patch_size=patch_size, max_patches=number_patches, random_state=random_state)
        if len(patch) == 0:
            raise ValueError("No patches extracted from image: {}".format(img_path))
        return patch.reshape((number_patches, np.prod(patch_size) * len(img.shape)))
    except Exception as e:
        print("Error loading or processing image {}".format(e))
        return None

patch_size = (4, 4)
number_patches = 20

patch_arr = []
for img_path in image_paths:
    patch = get_patches(img_path, 1, patch_size, number_patches,is_query=False)
    if patch is not None:
        patch_arr.append(patch)

def lbp_features(img, radius, sampling_pixels):
    """
    Compute Local Binary Pattern (LBP) features for an image.

    Parameters:
    - img (numpy.ndarray): Input image data.
    - radius (int): Radius for LBP computation.
    - sampling_pixels (int): Number of pixels to sample.

    Returns:
    - lbp_features (numpy.ndarray or None): Computed LBP features, or None if computation fails.
    """
    try:
        if (len(img.shape) > 2):
            img = img.astype(float)
            img = img[:,:,0]*0.3 + img[:,:,1]*0.59 + img[:,:,2]*0.11
        img = img.astype(np.uint8)
        i_min = np.min(img)
        i_max = np.max(img)
        if (i_max - i_min != 0):
            img = (img - i_min)/(i_max-i_min)
        lbp = feature.local_binary_pattern(img, sampling_pixels, radius, method="uniform")
        (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, sampling_pixels + 3), range=(0, sampling_pixels + 2))
        hist = hist.astype("float")
        hist /= (hist.sum() + 1e-6)
        return hist
    except Exception as e:
        print("Error loading lbp_features {}".format(e))
        return None

patch_arr = np.array(patch_arr, copy=True)
print(patch_arr.shape)
patch_arr = patch_arr.reshape((patch_arr.shape[0] * patch_arr.shape[1],
                               patch_size[0],patch_size[0],3))
performed_lbp = []
for pat in patch_arr:
        f = lbp_features(pat,5,8)
        performed_lbp.append(f)

performed_lbp = np.array(performed_lbp, copy=False)

number_features = 200
random_state = 1
kmeans_model = KMeans(n_clusters=number_features,
                      verbose=False,
                      init='random',
                      random_state=random_state,
                      n_init=3)
kmeans_model.fit(performed_lbp)

image_features = []
for i in range(len(image_paths)):
    y = kmeans_model.predict(performed_lbp[i*number_patches: (i*number_patches)+number_patches])
    hist_bof,_ = np.histogram(y, bins=range(number_features+1), density=True)
    image_features.append(hist_bof)

image_features = np.array(image_features, copy=False)


rand_int = np.random.randint(0,len(image_paths))
path_query = image_paths[rand_int]
query_patches = get_patches(path_query, random_state, patch_size, number_patches,is_query=True)
query_patches = np.array(query_patches, copy=False)


query_patches = query_patches.reshape((query_patches.shape[0],
                               patch_size[0],patch_size[0],3))
query_lbp = []
for pat in query_patches:
        f = lbp_features(pat,5,8)
        query_lbp.append(f)

query_lbp = np.array(query_lbp, copy=False)
y = kmeans_model.predict(query_lbp)
query_feats,_ = np.histogram(y, bins=range(number_features+1), density=True)
dists = []
for i in range(len(image_paths)):
    diq = np.sqrt(np.sum((image_features[i]-query_feats)**2))
    dists.append(diq)
k = 5
sorted_index = np.argsort(dists)[:k]

imgq = imageio.v2.imread(path_query)
img_with_zeros = cv2.copyMakeBorder(imgq, 0, 4, 0, 0, cv2.BORDER_CONSTANT, value=(255, 255, 255))
imgq = img_with_zeros[4:, :]
plt.imshow(imgq)
plt.figure(figsize=(12,8))
plt.title('Query'); plt.axis('on')

imgs = []
for idx,i in enumerate(sorted_index):
    imgs.append(imageio.v2.imread(image_paths[i]))
    plt.subplot(1,5,idx+1); plt.imshow(imgs[idx])
    plt.title(' Rank %s' % (idx+1)); plt.axis('on')